In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# 1. BASIC PREPARATIONS

In [3]:
weather_prepared = pd.read_csv('weather_data_prepared.csv')

In [4]:
weather_prepared.columns

Index(['region_id', 'date_value', 'time_value', 'dew', 'snow', 'temp',
       'precip', 'tempmax', 'tempmin', 'winddir', 'humidity', 'pressure',
       'windgust', 'moonphase', 'snowdepth', 'windspeed', 'cloudcover',
       'visibility', 'precipcover', 'datetime', 'precipprob_binary',
       'rain_bin', 'snow_bin', 'rain_snow_bin', 'sunrise_seconds',
       'sunset_seconds', 'daylight_duration_seconds'],
      dtype='object')

In [5]:
telegram_prepared = pd.read_csv('telegram_data30_3000.csv')

In [7]:
alarms_features_prepared = pd.read_csv('alarm_features_refined.csv') # with updated features, see report for detailed analysis

In [8]:
isw_prepared = pd.read_csv('isw_reports30_3000.csv') # with n_components = 30 and max_features = 3000, see report for detailed analysis

In [9]:
alarms_features_prepared['datetime'] = pd.to_datetime(alarms_features_prepared['datetime'], errors='coerce')
weather_prepared['datetime'] = pd.to_datetime(weather_prepared['datetime'], errors='coerce')
isw_prepared['date'] = pd.to_datetime(isw_prepared['date'], errors='coerce').dt.normalize()
telegram_prepared['date'] = pd.to_datetime(telegram_prepared['date'], errors='coerce').dt.normalize()

In [10]:
isw_prepared['date'] = isw_prepared['date'] - pd.Timedelta(days=1) # we use yesterday's reports for today's predictions

In [11]:
isw_prepared = isw_prepared.drop(index=0).reset_index(drop=True)

In [12]:
isw_prepared

,date,svd_comp_1,svd_comp_2,svd_comp_3,svd_comp_4,svd_comp_5,svd_comp_6,svd_comp_7,svd_comp_8,svd_comp_9,...,svd_comp_21,svd_comp_22,svd_comp_23,svd_comp_24,svd_comp_25,svd_comp_26,svd_comp_27,svd_comp_28,svd_comp_29,svd_comp_30
0,2022-02-24,0.311086,0.147366,0.176825,0.066852,0.154321,0.247315,0.174229,-0.154556,-0.212609,...,-0.048999,-0.032827,-0.025863,0.005627,0.018522,-0.023319,-0.026534,-0.015738,0.009063,-0.016790
1,2022-02-25,0.332181,0.197313,0.226641,0.073883,0.144300,0.297715,0.187955,-0.146418,-0.194939,...,-0.007026,-0.045585,0.009796,0.020144,0.030804,-0.057389,0.000073,0.002000,-0.014083,-0.046812
2,2022-02-26,0.337444,0.181902,0.225877,0.073972,0.162621,0.282900,0.168646,-0.166556,-0.183433,...,-0.008421,-0.005962,0.031128,0.004414,0.072174,-0.020013,0.022693,0.031146,-0.039998,-0.045724
3,2022-02-27,0.331243,0.180680,0.218240,0.089288,0.134707,0.274974,0.126843,-0.112598,-0.181635,...,0.015251,-0.006998,0.025542,0.019783,0.029482,-0.030549,0.051544,0.008276,-0.015945,-0.058782
4,2022-02-28,0.298613,0.214957,0.259313,0.098806,0.149972,0.289221,0.141712,-0.146604,-0.204224,...,-0.076646,-0.069538,-0.060807,0.030453,-0.065562,-0.121663,-0.082603,-0.116885,0.004056,-0.079697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2025-02-25,0.511094,-0.318700,0.160390,-0.164191,0.001623,0.013275,0.115133,0.217933,0.022499,...,-0.135295,-0.013439,0.026731,0.020502,-0.070420,0.000588,0.043042,0.010434,-0.005400,-0.040832
1088,2025-02-26,0.506097,-0.365712,0.193900,-0.197362,0.049416,0.014341,0.156342,0.217897,0.034938,...,-0.051647,0.034444,-0.005060,0.007726,-0.084801,0.022012,-0.008543,0.032491,0.017569,-0.027204
1089,2025-02-27,0.538457,-0.349700,0.168865,-0.176450,0.022700,-0.001643,0.098475,0.170720,0.030779,...,-0.010063,0.064180,0.008288,-0.006737,-0.054450,-0.027096,0.018672,0.021323,0.009913,-0.053991
1090,2025-02-28,0.518692,-0.320357,0.164328,-0.132340,0.001305,0.047174,0.092541,0.198658,0.015473,...,-0.058723,0.002736,0.034118,0.015842,-0.093537,-0.002935,0.049071,0.058113,0.015068,-0.060355


In [13]:
telegram_prepared['date'] = telegram_prepared['date'] - pd.Timedelta(days=1) # we use yesterday's reports for today's predictions

In [14]:
telegram_prepared = telegram_prepared.drop(index=0).reset_index(drop=True)

In [15]:
telegram_prepared

,date,svd2_comp_1,svd2_comp_2,svd2_comp_3,svd2_comp_4,svd2_comp_5,svd2_comp_6,svd2_comp_7,svd2_comp_8,svd2_comp_9,...,svd2_comp_21,svd2_comp_22,svd2_comp_23,svd2_comp_24,svd2_comp_25,svd2_comp_26,svd2_comp_27,svd2_comp_28,svd2_comp_29,svd2_comp_30
0,2022-02-24,0.233392,-0.240539,0.221081,-0.531773,-0.118241,0.124024,-0.006704,-0.015735,0.008527,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
1,2022-02-25,0.267848,-0.266961,0.236709,-0.576570,-0.123182,0.130856,-0.013946,-0.018136,0.023094,...,0.008065,-0.002319,0.007524,0.000623,-0.009618,-0.003655,-0.004840,-0.014712,-0.036364,-0.001958
2,2022-02-26,0.257742,-0.265058,0.236697,-0.596808,-0.136509,0.136100,-0.010833,-0.024805,0.020053,...,-0.004286,-0.003724,0.008926,-0.005995,-0.004563,-0.009058,-0.002876,-0.020473,-0.040288,0.013216
3,2022-02-27,0.241693,-0.259066,0.233190,-0.587735,-0.126220,0.144992,-0.002781,-0.011495,0.020685,...,-0.006940,-0.002218,0.000779,0.011256,-0.017818,0.005484,0.008346,-0.007076,-0.044399,-0.015507
4,2022-02-28,0.268409,-0.264711,0.232078,-0.570876,-0.105494,0.122857,-0.003485,-0.015003,0.018080,...,-0.003406,-0.025714,0.016366,-0.002192,-0.012288,-0.010797,0.000698,-0.020169,-0.047865,-0.011654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2025-02-25,0.344489,0.311677,0.083349,-0.057610,0.236084,0.079052,-0.013511,0.050764,0.110069,...,-0.023014,0.044358,0.099546,-0.099201,0.075443,-0.017159,-0.045433,0.046930,-0.074921,0.000270
1092,2025-02-26,0.280508,0.256076,0.083980,-0.086677,0.231610,0.105985,-0.024709,0.076428,0.163577,...,-0.006401,-0.000481,0.108406,-0.110323,0.078858,-0.051852,0.006235,0.028782,-0.060498,0.020732
1093,2025-02-27,0.393390,0.282169,0.094768,-0.089410,0.199744,0.085729,-0.016792,0.060891,0.084542,...,0.062051,-0.002300,-0.076323,0.010075,0.099644,0.078373,-0.139530,-0.058867,-0.052389,0.076064
1094,2025-02-28,0.359724,0.280275,0.076676,-0.072708,0.195589,0.095542,-0.029812,0.054710,0.087973,...,-0.034000,-0.039273,0.031171,-0.066673,0.069704,-0.101913,0.030359,-0.013072,0.007667,0.040670


In [16]:
# 2. DATE CHECK

In [17]:
existing_dates_weather = pd.Series(weather_prepared['datetime'].unique())

In [18]:
start_range = datetime(2022, 2, 24)
end_range = datetime(2025, 3, 1)

In [19]:
full_range = pd.date_range(start=start_range, end=end_range, freq='h')

In [20]:
missing_dates_weather = full_range.difference(existing_dates_weather)

In [21]:
missing_dates_weather

DatetimeIndex(['2022-03-27 03:00:00', '2023-03-26 03:00:00',
               '2024-03-31 03:00:00'],
              dtype='datetime64[ns]', freq=None)

In [22]:
existing_dates_alarms = pd.Series(alarms_features_prepared['datetime'].unique())

In [23]:
missing_dates_alarm = full_range.difference(existing_dates_alarms)

In [24]:
missing_dates_alarm

DatetimeIndex([], dtype='datetime64[ns]', freq='h')

In [25]:
# 3. MERGING

In [26]:
merged_v1 = pd.merge(
    alarms_features_prepared,
    weather_prepared,
    left_on=['region_id', 'datetime'],
    right_on=['region_id', 'datetime'],
    how='left'
)

In [27]:
merged_v1['date_for_merge'] = merged_v1['datetime'].dt.normalize()

In [28]:
merged_v1.isna().sum()

region_id                                             0
time_since_last_alarm_end_minutes_at_start_of_day     0
total_alarm_minutes_yesterday                         0
alarms_started_yesterday                              0
alarms_started_trend                                  0
was_alarm_active_end_of_yesterday                     0
is_alarm_active_lag_7d                                0
avg_daily_alarm_minutes_last_7_days                   0
is_alarm_active                                       0
datetime                                              0
date_value                                           72
time_value                                           72
dew                                                  72
snow                                                 72
temp                                                 72
precip                                               72
tempmax                                              72
tempmin                                         

In [29]:
alarm_keys = set(zip(alarms_features_prepared['region_id'], alarms_features_prepared['datetime']))
weather_keys = set(zip(weather_prepared['region_id'], weather_prepared['datetime']))

missing_keys = alarm_keys - weather_keys

In [30]:
# missing on the dates around the end of March each year (daylight saving time)
# this is not a data loss during the transformation - in the original dataset with
# historical data for all days of daylight saving time, the weather values for 03:00:00 are missing

missing_keys

{(1, Timestamp('2022-03-27 03:00:00')),
 (1, Timestamp('2023-03-26 03:00:00')),
 (1, Timestamp('2024-03-31 03:00:00')),
 (2, Timestamp('2022-03-27 03:00:00')),
 (2, Timestamp('2023-03-26 03:00:00')),
 (2, Timestamp('2024-03-31 03:00:00')),
 (3, Timestamp('2022-03-27 03:00:00')),
 (3, Timestamp('2023-03-26 03:00:00')),
 (3, Timestamp('2024-03-31 03:00:00')),
 (4, Timestamp('2022-03-27 03:00:00')),
 (4, Timestamp('2023-03-26 03:00:00')),
 (4, Timestamp('2024-03-31 03:00:00')),
 (5, Timestamp('2022-03-27 03:00:00')),
 (5, Timestamp('2023-03-26 03:00:00')),
 (5, Timestamp('2024-03-31 03:00:00')),
 (6, Timestamp('2022-03-27 03:00:00')),
 (6, Timestamp('2023-03-26 03:00:00')),
 (6, Timestamp('2024-03-31 03:00:00')),
 (7, Timestamp('2022-03-27 03:00:00')),
 (7, Timestamp('2023-03-26 03:00:00')),
 (7, Timestamp('2024-03-31 03:00:00')),
 (8, Timestamp('2022-03-27 03:00:00')),
 (8, Timestamp('2023-03-26 03:00:00')),
 (8, Timestamp('2024-03-31 03:00:00')),
 (9, Timestamp('2022-03-27 03:00:00')),


In [31]:
final_df = pd.merge(
    merged_v1,
    isw_prepared,
    left_on='date_for_merge',
    right_on='date',
    how='left'
)

final_df = final_df.drop(columns=['date'])

In [32]:
final_df2 = pd.merge(
    final_df,
    telegram_prepared,
    left_on='date_for_merge',
    right_on='date',
    how='left'
)


In [33]:
final_df2

,region_id,time_since_last_alarm_end_minutes_at_start_of_day,total_alarm_minutes_yesterday,alarms_started_yesterday,alarms_started_trend,was_alarm_active_end_of_yesterday,is_alarm_active_lag_7d,avg_daily_alarm_minutes_last_7_days,is_alarm_active,datetime,...,svd2_comp_21,svd2_comp_22,svd2_comp_23,svd2_comp_24,svd2_comp_25,svd2_comp_26,svd2_comp_27,svd2_comp_28,svd2_comp_29,svd2_comp_30
0,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
1,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 01:00:00,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
2,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 02:00:00,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
3,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 03:00:00,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
4,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 04:00:00,...,-0.008076,-0.012690,0.002526,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634747,25,357.616667,363.866667,2,1,True,True,511.657143,0,2025-03-01 19:00:00,...,-0.010122,-0.004397,0.078465,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464
634748,25,357.616667,363.866667,2,1,True,True,511.657143,0,2025-03-01 20:00:00,...,-0.010122,-0.004397,0.078465,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464
634749,25,357.616667,363.866667,2,1,True,True,511.657143,1,2025-03-01 21:00:00,...,-0.010122,-0.004397,0.078465,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464
634750,25,357.616667,363.866667,2,1,True,True,511.657143,1,2025-03-01 22:00:00,...,-0.010122,-0.004397,0.078465,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464


In [34]:
final_df = final_df2

In [35]:
final_df = final_df.sort_values(by=['datetime', 'region_id']).reset_index(drop=True)

In [36]:
# we see gaps in weather (described above) and gaps in isw data (described in isw_vectorization.ipynb)
# we'll work through these gaps in the next step

final_df.isna().sum()

region_id                                               0
time_since_last_alarm_end_minutes_at_start_of_day       0
total_alarm_minutes_yesterday                           0
alarms_started_yesterday                                0
alarms_started_trend                                    0
                                                     ... 
svd2_comp_26                                         3456
svd2_comp_27                                         3456
svd2_comp_28                                         3456
svd2_comp_29                                         3456
svd2_comp_30                                         3456
Length: 97, dtype: int64

In [37]:
# weather nans
weather_cols = weather_prepared.columns.difference(['region_id', 'datetime']).tolist()
final_df[weather_cols] = final_df.groupby('region_id')[weather_cols].ffill()

# svd nans
svd_cols = [col for col in isw_prepared.columns if col.startswith('svd_comp_')]
final_df[svd_cols] = final_df[svd_cols].ffill()

# svd nans
svd2_cols = [col for col in telegram_prepared.columns if col.startswith('svd2_comp_')]
final_df[svd2_cols] = final_df[svd2_cols].ffill()

# final check
final_df.isna().sum()

region_id                                            0
time_since_last_alarm_end_minutes_at_start_of_day    0
total_alarm_minutes_yesterday                        0
alarms_started_yesterday                             0
alarms_started_trend                                 0
                                                    ..
svd2_comp_26                                         0
svd2_comp_27                                         0
svd2_comp_28                                         0
svd2_comp_29                                         0
svd2_comp_30                                         0
Length: 97, dtype: int64

In [38]:
final_df['datetime'] = pd.to_datetime(final_df['datetime'], errors='coerce')
final_df['hour_of_day'] = final_df['datetime'].dt.hour
final_df['day_of_week'] = final_df['datetime'].dt.dayofweek
final_df['month'] = final_df['datetime'].dt.month

In [39]:
final_df = final_df.copy()

In [40]:
final_df

,region_id,time_since_last_alarm_end_minutes_at_start_of_day,total_alarm_minutes_yesterday,alarms_started_yesterday,alarms_started_trend,was_alarm_active_end_of_yesterday,is_alarm_active_lag_7d,avg_daily_alarm_minutes_last_7_days,is_alarm_active,datetime,...,svd2_comp_24,svd2_comp_25,svd2_comp_26,svd2_comp_27,svd2_comp_28,svd2_comp_29,svd2_comp_30,hour_of_day,day_of_week,month
0,1,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499,0,3,2
1,2,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499,0,3,2
2,3,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499,0,3,2
3,4,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499,0,3,2
4,5,509629.350000,0.000000,0,0,False,False,0.000000,0,2022-02-24 00:00:00,...,-0.003743,-0.006480,-0.021615,0.011399,-0.012310,-0.046835,0.000499,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634747,21,63.300000,254.316667,5,3,True,True,294.380952,1,2025-03-01 23:00:00,...,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464,23,5,3
634748,22,2602.533333,0.000000,0,-1,False,False,99.285714,0,2025-03-01 23:00:00,...,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464,23,5,3
634749,23,377.200000,25.616667,2,2,False,False,17.128571,0,2025-03-01 23:00:00,...,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464,23,5,3
634750,24,5312.016667,0.000000,0,0,False,False,16.928571,0,2025-03-01 23:00:00,...,-0.071547,0.054663,-0.000527,0.048247,-0.060574,0.023855,-0.000464,23,5,3


In [41]:
final_df.to_csv('final_df_merged.csv', index=False)